In [1]:
import torch

In [2]:
from operator import itemgetter, attrgetter

In [3]:
words = open('names.txt','r').read().splitlines()

In [4]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [5]:
#finding min len of word in list of words
min([len(w) for w in words])

2

In [6]:
#finding max len of word in list of words
max([len(w) for w in words])

15

In [7]:
b = {}
for w in words:
    chs = ['<S'] + list(w) + ['E>']
    for ch1,ch2 in zip(chs,chs[1:]):
        bigram = (ch1,ch2)
        b[bigram] = b.get(bigram,0) + 1
        #print(b)
        #print(ch1,ch2)

In [8]:
sorted(b.items(),key = itemgetter(1), reverse = True)

[(('n', 'E>'), 6763),
 (('a', 'E>'), 6640),
 (('a', 'n'), 5438),
 (('<S', 'a'), 4410),
 (('e', 'E>'), 3983),
 (('a', 'r'), 3264),
 (('e', 'l'), 3248),
 (('r', 'i'), 3033),
 (('n', 'a'), 2977),
 (('<S', 'k'), 2963),
 (('l', 'e'), 2921),
 (('e', 'n'), 2675),
 (('l', 'a'), 2623),
 (('m', 'a'), 2590),
 (('<S', 'm'), 2538),
 (('a', 'l'), 2528),
 (('i', 'E>'), 2489),
 (('l', 'i'), 2480),
 (('i', 'a'), 2445),
 (('<S', 'j'), 2422),
 (('o', 'n'), 2411),
 (('h', 'E>'), 2409),
 (('r', 'a'), 2356),
 (('a', 'h'), 2332),
 (('h', 'a'), 2244),
 (('y', 'a'), 2143),
 (('i', 'n'), 2126),
 (('<S', 's'), 2055),
 (('a', 'y'), 2050),
 (('y', 'E>'), 2007),
 (('e', 'r'), 1958),
 (('n', 'n'), 1906),
 (('y', 'n'), 1826),
 (('k', 'a'), 1731),
 (('n', 'i'), 1725),
 (('r', 'e'), 1697),
 (('<S', 'd'), 1690),
 (('i', 'e'), 1653),
 (('a', 'i'), 1650),
 (('<S', 'r'), 1639),
 (('a', 'm'), 1634),
 (('l', 'y'), 1588),
 (('<S', 'l'), 1572),
 (('<S', 'c'), 1542),
 (('<S', 'e'), 1531),
 (('j', 'a'), 1473),
 (('r', 'E>'), 137

In [9]:
#lets keep this in 2d array
# rows are 1st ch and col are 2nd ch

In [10]:
N = torch.zeros((27,27), dtype = torch.int32) # 26 alphabets plus 2 special start and end tokens

In [11]:
chars = sorted(set(''.join(words)))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [12]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1,ix2] +=1

In [13]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.figure(figsize=(16,16))
# plt.imshow(N, cmap='Blues')
# for i in range(28):
#     for j in range(28):
#         chstr = itos[i] + itos[j]
#         plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
#         plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
# plt.axis('off');

In [14]:
N[0]

tensor([   0, 4410, 1306, 1542, 1690, 1531,  417,  669,  874,  591, 2422, 2963,
        1572, 2538, 1146,  394,  515,   92, 1639, 2055, 1308,   78,  376,  307,
         134,  535,  929], dtype=torch.int32)

In [15]:
N[0,2] / N[0].sum()

tensor(0.0408)

In [16]:
p = N[0].float()
p = p / p.sum() #normalize
p

tensor([0.0000, 0.1377, 0.0408, 0.0481, 0.0528, 0.0478, 0.0130, 0.0209, 0.0273,
        0.0184, 0.0756, 0.0925, 0.0491, 0.0792, 0.0358, 0.0123, 0.0161, 0.0029,
        0.0512, 0.0642, 0.0408, 0.0024, 0.0117, 0.0096, 0.0042, 0.0167, 0.0290])

In [17]:
p.sum()

tensor(1.)

In [18]:
g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itos[ix]

'm'

In [19]:
P = (N+1).float()
P = P/ P.sum(axis = 1, keepdim = True)

In [20]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out = []
    ix = 0
    while True:
#         p = N[ix].float()
#         p = p/p.sum()
        p = P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        if ix == 0:
            break
        out.append(itos[ix])

    print(''.join(out))

mor
axx
minaymoryles
kondlaisah
anchshizarie


In [21]:
log_likely_hood = 0.0
n = 0
for w in ["asheeshjq"]:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        p = P[ix1,ix2]
        logp = torch.log(p)
        log_likely_hood += logp
        n+=1
        print(f'{ch1}{ch2} : {p:.2f} {logp: .2f}')
print(f'{log_likely_hood=}')
nll = -log_likely_hood
print(f'{nll=}')
print(f'normalized nll: {nll/n}')

.a : 0.14 -1.98
as : 0.03 -3.41
sh : 0.16 -1.84
he : 0.09 -2.43
ee : 0.06 -2.78
es : 0.04 -3.17
sh : 0.16 -1.84
hj : 0.00 -6.64
jq : 0.00 -7.98
q. : 0.10 -2.33
log_likely_hood=tensor(-34.4082)
nll=tensor(34.4082)
normalized nll: 3.4408175945281982


**Approach 2 Using NN**

In [22]:
#creating training data set of bigrams(x,y)

xs, ys = [] , []
for w in words[:1]:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [23]:
xs

tensor([ 0,  5, 13, 13,  1])

In [24]:
ys

tensor([ 5, 13, 13,  1,  0])

In [25]:
#encoding the values
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=27).float()

In [26]:
xenc

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [27]:
xenc.shape

torch.Size([5, 27])

In [28]:
xenc.dtype

torch.float32

In [32]:
#build a single neuron

W = torch.randn((27,27))

xenc @ W

tensor([[ 0.2993, -0.9032,  1.1147,  0.8408,  0.9417,  0.1955,  0.6066, -0.8068,
         -0.9951, -0.7316,  0.1055, -0.1907,  0.2367,  0.9110, -0.0196,  0.5596,
          1.0089,  0.2613,  0.4686, -0.7418, -1.4561, -0.1257,  1.8790,  1.8538,
         -0.1056, -1.2839, -0.7695],
        [ 0.6965,  1.3609,  0.5710, -1.1756, -0.2038,  0.7772, -1.1281,  1.4317,
         -0.0371, -0.5092, -0.9179,  1.6242, -1.0693,  2.7455, -0.3422,  0.0594,
         -0.2524, -1.1267,  0.9537,  0.5177,  0.0783, -0.9880,  0.9356,  1.7065,
          1.0721,  0.0322, -0.1065],
        [ 0.2553,  0.0682,  0.1819,  0.5261, -0.9319, -0.0367, -0.9463,  1.0085,
         -2.8253,  1.5701,  1.0760,  0.4798, -0.2380, -0.3560, -1.3579, -0.2161,
          0.2742,  0.7012, -1.0731, -3.1951,  0.4655, -0.3705, -2.3923,  0.5639,
         -0.7568,  0.7133,  1.6138],
        [ 0.2553,  0.0682,  0.1819,  0.5261, -0.9319, -0.0367, -0.9463,  1.0085,
         -2.8253,  1.5701,  1.0760,  0.4798, -0.2380, -0.3560, -1.3579, -0.2161

In [33]:
logits = xenc @ W # log-counts
counts = logits.exp() #equivalent to N
probs = counts / counts.sum(1, keepdims = True)

In [37]:
probs[0].sum()

tensor(1.)

In [38]:
probs[0]

tensor([0.0305, 0.0092, 0.0689, 0.0524, 0.0579, 0.0275, 0.0414, 0.0101, 0.0084,
        0.0109, 0.0251, 0.0187, 0.0286, 0.0562, 0.0222, 0.0395, 0.0620, 0.0293,
        0.0361, 0.0108, 0.0053, 0.0199, 0.1479, 0.1443, 0.0203, 0.0063, 0.0105])

In [40]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g)

In [41]:
#forward pass
xenc = F.one_hot(xs, num_classes=27).float() #encode
logits = xenc @ W # log-counts
counts = logits.exp() #equivalent to N
probs = counts / counts.sum(1, keepdims = True)

In [43]:
nlls = torch.zeros(5)
for i in range(5):
  # i-th bigram:
  x = xs[i].item() # input character index
  y = ys[i].item() # label character index
  print('--------')
  print(f'bigram example {i+1}: {itos[x]}{itos[y]} (indexes {x},{y})')
  print('input to the neural net:', x)
  print('output probabilities from the neural net:', probs[i])
  print('label (actual next character):', y)
  p = probs[i, y]
  print('probability assigned by the net to the the correct character:', p.item())
  logp = torch.log(p)
  print('log likelihood:', logp.item())
  nll = -logp
  print('negative log likelihood:', nll.item())
  nlls[i] = nll

print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())

--------
bigram example 1: .e (indexes 0,5)
input to the neural net: 0
output probabilities from the neural net: tensor([0.0607, 0.0100, 0.0123, 0.0042, 0.0168, 0.0123, 0.0027, 0.0232, 0.0137,
        0.0313, 0.0079, 0.0278, 0.0091, 0.0082, 0.0500, 0.2378, 0.0603, 0.0025,
        0.0249, 0.0055, 0.0339, 0.0109, 0.0029, 0.0198, 0.0118, 0.1537, 0.1459])
label (actual next character): 5
probability assigned by the net to the the correct character: 0.01228625513613224
log likelihood: -4.399273872375488
negative log likelihood: 4.399273872375488
--------
bigram example 2: em (indexes 5,13)
input to the neural net: 5
output probabilities from the neural net: tensor([0.0290, 0.0796, 0.0248, 0.0521, 0.1989, 0.0289, 0.0094, 0.0335, 0.0097,
        0.0301, 0.0702, 0.0228, 0.0115, 0.0181, 0.0108, 0.0315, 0.0291, 0.0045,
        0.0916, 0.0215, 0.0486, 0.0300, 0.0501, 0.0027, 0.0118, 0.0022, 0.0472])
label (actual next character): 13
probability assigned by the net to the the correct character: 0.

In [44]:
#optimization

In [52]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad = True)

In [69]:
#forward pass
xenc = F.one_hot(xs, num_classes=27).float() #encode
logits = xenc @ W # log-counts
counts = logits.exp() #equivalent to N
probs = counts / counts.sum(1, keepdims = True)
ypred = probs[torch.arange(5),ys] #these are the predictions for range in 5 indices and ys 
loss = -ypred.log().mean() #negative log likelihood is the loss
loss

tensor(3.6693, grad_fn=<NegBackward>)

In [67]:
#backward pass
W.grad = None
loss.backward()

In [68]:
W.data += -0.1*W.grad

In [73]:
#rearranging everything together

In [74]:
#creating training data set of bigrams(x,y)
xs, ys = [] , []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1,ch2 in zip(chs,chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print("number of examples", num)

number of examples 228146


In [77]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad = True)

for i in range(50):
    #forward pass
    xenc = F.one_hot(xs, num_classes=27).float() #encode
    logits = xenc @ W # log-counts
    counts = logits.exp() #equivalent to N
    probs = counts / counts.sum(1, keepdims = True)
    ypred = probs[torch.arange(num),ys] #these are the predictions for range in 5 indices and ys 
    loss = -ypred.log().mean() #negative log likelihood is the loss
    print(loss.item())
    #backward pass
    W.grad = None
    loss.backward()

    #updating weights using grads
    W.data += -50*W.grad

3.758953332901001
3.371100664138794
3.154043197631836
3.020373582839966
2.927711248397827
2.8604023456573486
2.8097290992736816
2.7701022624969482
2.7380728721618652
2.711496353149414
2.6890029907226562
2.6696884632110596
2.6529300212860107
2.638277292251587
2.6253881454467773
2.613990545272827
2.60386323928833
2.5948216915130615
2.5867116451263428
2.579403877258301
2.572789192199707
2.5667762756347656
2.5612878799438477
2.5562589168548584
2.551633834838867
2.547366142272949
2.543415069580078
2.5397486686706543
2.5363364219665527
2.5331544876098633
2.5301806926727295
2.5273969173431396
2.5247862339019775
2.522334575653076
2.520028829574585
2.517857789993286
2.515810966491699
2.513878345489502
2.512052059173584
2.510324001312256
2.5086867809295654
2.5071346759796143
2.5056614875793457
2.5042612552642822
2.502929210662842
2.5016613006591797
2.5004520416259766
2.4992988109588623
2.498197317123413
2.497144937515259


In [82]:
#sampling from neural net
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out = []
    ix = 0
    while True:
#         p = N[ix].float()
#         p = p/p.sum()
#         p = P[ix]
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float() #encode
        logits = xenc @ W # log-counts
        counts = logits.exp() #equivalent to N
        probs = counts / counts.sum(1, keepdims = True)
        ix = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
        if ix == 0:
            break
        out.append(itos[ix])

    print(''.join(out))

mor
axx
minaymoryles
kondmaisah
anchthizarie
